# CDAT Migration Regression Testing Notebook (`.nc` files)

This notebook is used to perform regression testing between the development and
production versions of a diagnostic set.

## How it works

It compares the relative differences (%) between ref and test variables between
the dev and `main` branches.

## How to use

PREREQUISITE: The diagnostic set's netCDF stored in `.json` files in two directories
(dev and `main` branches).

1. Make a copy of this notebook under `auxiliary_tools/cdat_regression_testing/<DIR_NAME>`.
2. Run `mamba create -n cdat_regression_test -y -c conda-forge "python<3.12" xarray netcdf4 dask pandas matplotlib-base ipykernel`
3. Run `mamba activate cdat_regression_test`
4. Update `SET_DIR` and `SET_NAME` in the copy of your notebook.
5. Run all cells IN ORDER.
6. Review results for any outstanding differences (>=1e-5 relative tolerance).
   - Debug these differences (e.g., bug in metrics functions, incorrect variable references, etc.)


## Setup Code


In [7]:
import glob
from collections import defaultdict

import numpy as np
import xarray as xr

# TODO: Update SET_NAME and SET_DIR
SET_NAME = "zonal_mean_xy"
SET_DIR = "debug-654-zonal_mean_xy"

DEV_PATH = f"/global/cfs/projectdirs/e3sm/e3sm_diags_cdat_test/{SET_DIR}/{SET_NAME}/**"
MAIN_PATH = f"/global/cfs/projectdirs/e3sm/e3sm_diags_cdat_test/main/{SET_NAME}/**"

DEV_GLOB = sorted(glob.glob(DEV_PATH + "/*.nc"))
MAIN_GLOB = sorted(glob.glob(MAIN_PATH + "/*.nc"))

if len(DEV_GLOB) == 0 or len(MAIN_GLOB) == 0:
    raise IOError("No files found at DEV_PATH and/or MAIN_PATH.")

if len(DEV_GLOB) != len(MAIN_GLOB):
    # raise IOError(f"Number of files do not match at DEV_PATH ({len(DEV_GLOB)}) and MAIN_PATH {len(MAIN_GLOB)}.")
    print(
        f"Number of files do not match at DEV_PATH ({len(DEV_GLOB)}) and MAIN_PATH {len(MAIN_GLOB)}."
    )

In [8]:
missing_count = 0
for filepath_main in MAIN_GLOB:
    filepath_dev = filepath_main.replace("main", SET_DIR)
    try:
        ds = xr.open_dataset(filepath_dev)
    except OSError:
        print(f"No file found to compare with {filepath_main}!")
        missing_count += 1
print(f"Number of files missing: {missing_count}")

Number of files missing: 0


## 1. Compare the netCDF files between branches

- Compare "ref" and "test" files
- "diff" files are ignored because getting relative diffs for these does not make sense (relative diff will be above tolerance)


In [9]:
# We are mainly focusing on relative tolerance here (in percentage terms).
atol = 0
rtol = 1e-5

for filepath_dev in DEV_GLOB:
    if "test.nc" in filepath_dev or "ref.nc" in filepath_dev:
        filepath_main = filepath_dev.replace(SET_DIR, "main")
        print("Comparing:")
        print(filepath_dev, "\n", filepath_main)
        ds1 = xr.open_dataset(filepath_dev)
        ds2 = xr.open_dataset(filepath_main)

        var_key = filepath_dev.split("-")[-3]
        # for 3d vars such as T-200
        var_key.isdigit()
        if var_key.isdigit():
            var_key = filepath_dev.split("-")[-4]

        print("var_key", var_key)
        try:
            np.testing.assert_allclose(
                ds1[var_key].values,
                ds2[var_key].values,
                atol=atol,
                rtol=rtol,
            )
        except AssertionError as e:
            print(e)
        else:
            print(f"   * All close and within relative tolerance ({rtol})")

Comparing:
/global/cfs/projectdirs/e3sm/e3sm_diags_cdat_test/debug-654-zonal_mean_xy/zonal_mean_xy/CERES-EBAF-TOA-v4.1/ceres_ebaf_toa_v4.1-ALBEDO-ANN-global_ref.nc 
 /global/cfs/projectdirs/e3sm/e3sm_diags_cdat_test/main/zonal_mean_xy/CERES-EBAF-TOA-v4.1/ceres_ebaf_toa_v4.1-ALBEDO-ANN-global_ref.nc
var_key ALBEDO
   * All close and within relative tolerance (1e-05)
Comparing:
/global/cfs/projectdirs/e3sm/e3sm_diags_cdat_test/debug-654-zonal_mean_xy/zonal_mean_xy/CERES-EBAF-TOA-v4.1/ceres_ebaf_toa_v4.1-ALBEDO-ANN-global_test.nc 
 /global/cfs/projectdirs/e3sm/e3sm_diags_cdat_test/main/zonal_mean_xy/CERES-EBAF-TOA-v4.1/ceres_ebaf_toa_v4.1-ALBEDO-ANN-global_test.nc
var_key ALBEDO
   * All close and within relative tolerance (1e-05)
Comparing:
/global/cfs/projectdirs/e3sm/e3sm_diags_cdat_test/debug-654-zonal_mean_xy/zonal_mean_xy/CERES-EBAF-TOA-v4.1/ceres_ebaf_toa_v4.1-ALBEDO-JJA-global_ref.nc 
 /global/cfs/projectdirs/e3sm/e3sm_diags_cdat_test/main/zonal_mean_xy/CERES-EBAF-TOA-v4.1/ceres_e

### Results

- Some remaining issues for mismatched `nan` locations and large relative diffs that will be resolved into another PR.
